In [ ]:
import holodeck as holo
from holodeck import single_sources, utils, plot, detstats, anisotropy
from holodeck.constants import YR

import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py



# Set Up

## Read in library

ss16 has 10 loudest, definitely will replace this with a better library

In [ ]:
sspath = '/Users/emigardiner/GWs/holodeck/output/2023-05-16-mbp-ss19_uniform05A_n1000_r50_d20_f30_l2000_p0/'
hdfname = sspath+'ss_lib.hdf5'
ssfile = h5py.File(hdfname, 'r')
print(list(ssfile.keys()))
hc_ss = ssfile['hc_ss'][...]
hc_bg = ssfile['hc_bg'][...]
fobs = ssfile['fobs'][:]
dfobs = ssfile['dfobs'][:]
ssfile.close()

shape = hc_ss.shape
nsamps, nfreqs, nreals, nloudest = shape[0], shape[1], shape[2], shape[3]
print('N,F,R,L =', nsamps, nfreqs, nreals, nloudest)


## Find Best Samples

In [ ]:
hc_ref15_10yr = 11.2*10**-15 
nsort, fidx, hc_tt, hc_ref15 = detstats.rank_samples(hc_ss, hc_bg, fobs, hc_ref=hc_ref15_10yr, ret_all=True)
nbest = 100
print(hc_ref15)


## Load/Calculate pixel strains and harmonic coefficients

In [ ]:
npix = anisotropy.NPIX
lmax = anisotropy.LMAX

Cl_best = np.zeros((nbest, nfreqs, nreals, lmax+1 ))
moll_hc_best = np.zeros((nbest, nfreqs, nreals, npix))
for nn in range(nbest):
    print('on nn=%d out of nbest=%d' % (nn,nbest))
    moll_hc_best[nn,...], Cl_best[nn,...] = anisotropy.sph_harm_from_hc(hc_ss[nsort[nn]], 
                                            hc_bg[nsort[nn]])

In [ ]:
# np.savez(sspath+'sph_harmonics.npz', nsort=nsort, fidx=fidx, hc_tt=hc_tt, hc_ref15=hc_ref15, ss_shape=shape,
#          moll_hc_best=moll_hc_best, Cl_best=Cl_best, nside=nside, lmax=lmax, fobs=fobs)

sph_harm_file = np.load(sspath+'sph_harmonics.npz')

# load ss info
shape = sph_harm_file['ss_shape']
nsamps, nfreqs, nreals, nloudest = shape[0], shape[1], shape[2], shape[3]
fobs = sph_harm_file['fobs']

# load ranking info
nsort = sph_harm_file['nsort']
fidx = sph_harm_file['fidx']
hc_tt = sph_harm_file['hc_tt']
hc_ref15 = sph_harm_file['hc_ref15']

# load harmonics info
nside = sph_harm_file['nside']
lmax  = sph_harm_file['lmax']
moll_hc_best = sph_harm_file['moll_hc_best']
Cl_best = sph_harm_file['Cl_best']
nbest = len(moll_hc_best)

sph_harm_file.close()

### Plot Results

In [ ]:
lvals = np.arange(lmax+1)
fobs_nHz = fobs*10**9
fobs_yrs = fobs*YR
ff_labels = (0,9,19,29,39)
nshow = 10

#### Plot_Cl_medians

In [ ]:
def plot_Cl_medians(lvals, Cl_best, lmax):
    fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell$', xscale='linear')

    xx = lvals # (l)
    yy = Cl_best # (B,F,R,l)
    yy = np.median(yy, axis=2) # (B,F,l)

    colors = cm.rainbow(np.linspace(1, 0, nfreqs+1))
    for ff in (np.arange(1,nfreqs,8)):
        label= ('$f$ = %.2f nHz = %.2f /yr' % (fobs_nHz[ff], fobs_yrs[ff]))
        ax.plot(xx, np.median(yy[:,ff,:], axis=0), color=colors[ff], label=label,
                linewidth=3)
        for pp in [50,]:
            percs = pp/2
            percs = [50-percs, 50+percs]
            ax.fill_between(xx, *np.percentile(yy[:,ff,:], percs, axis=0), alpha=0.25, color=colors[ff])
        for bb in range(nshow):
            ax.plot(xx, yy[bb,ff,:], color=colors[ff], linestyle=':', alpha=0.75)

    ax.legend()
    ax.set_title('50%% confidence intervals of the %d best samples \nusing realizations medians, lmax=%d'
                % (nbest, lmax))
    return fig

fig = plot_Cl_medians(lvals, Cl_best, lmax)

In [ ]:
fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell$', xscale='linear')

xx = lvals # (l)
yy = Cl_best # (B,F,R,l)
yy = np.median(yy, axis=2) # (B,F,l)

# choose a frequency 
ff = fidx # f=1/yr
label= ('$f$ = %.2f nHz = %.2f /yr' % (fobs_nHz[ff], fobs_yrs[ff]))
yy = yy[:,ff,:] # (B,l)

colors = cm.rainbow(np.linspace(1, 0, 31))
ax.plot(xx, np.median(yy, axis=0), color=colors[ff], label=label, lw='3')
for pp in [50, 98]:
    percs = pp/2
    percs = [50-percs, 50+percs]
    ax.fill_between(xx, *np.percentile(yy, percs, axis=0), alpha=0.25, color=colors[ff])

colors = cm.rainbow(np.linspace(0,1,nshow))
for bb in range(nshow):
    ax.plot(xx, yy[bb,:], color=colors[bb], linestyle=':', alpha=0.75)
ax.legend()
ax.set_title('50%% and 98%% Confidence Intervals of the %d Best Samples, \nusing median among realizations, lmax=%d' % (nbest, lmax))

#### plot_ClC0_medians

In [ ]:
def plot_ClC0_medians(lvals, Cl_best, lmax):
    fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell/C_0$', xscale='linear')

    xx = lvals[1:] # (l)
    yy = Cl_best[:,:,:,1:]/Cl_best[:,:,:,0,np.newaxis] # (B,F,R,l)
    yy = np.median(yy, axis=2) # (F,R,l)

    colors = cm.rainbow(np.linspace(1, 0, nfreqs+1))
    for ff in (np.arange(0,nfreqs,8)):
        label= ('$f$ = %.2f nHz = %.2f /yr' % (fobs_nHz[ff], fobs_yrs[ff]))
        ax.plot(xx, np.median(yy[:,ff,:], axis=0), color=colors[ff], label=label,
                linewidth=4)
        for pp in [50,]:
            percs = pp/2
            percs = [50-percs, 50+percs]
            ax.fill_between(xx, *np.percentile(yy[:,ff,:], percs, axis=0), alpha=0.25, color=colors[ff],
                            linestyle='solid', linewidth=3)
        for bb in range(nshow):
            ax.plot(xx, yy[bb,ff,:], color=colors[ff], linestyle=':', alpha=0.75)

    fig.legend(ncol=3, bbox_to_anchor=(-.01,-.27), 
            bbox_transform=ax.transAxes, loc='lower left')
    ax.set_title('50%% confidence intervals of the %d best samples \nusing realizations medians, lmax=%d'
                % (nbest, lmax))
    fig.tight_layout()
    return fig

fig = plot_ClC0_medians(lvals, Cl_best, lmax)

In [ ]:
fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell/C_0$', xscale='linear')

xx = lvals[1:] # (l)
yy = Cl_best[:,:,:,1:]/Cl_best[:,:,:,0,np.newaxis] # (B,F,R,l)
yy = np.median(yy, axis=2) # (B,F,l)

# choose a frequency 
ff = fidx # f=1/yr
label= ('$f$ = %.2f nHz = %.2f /yr' % (fobs_nHz[ff], fobs_yrs[ff]))
yy = yy[:,ff,:] # (B,l)

colors = cm.rainbow(np.linspace(1, 0, 31))
ax.plot(xx, np.median(yy, axis=0), color=colors[ff], label=label, lw=4)
for pp in [50, 98]:
    percs = pp/2
    percs = [50-percs, 50+percs]
    ax.fill_between(xx, *np.percentile(yy, percs, axis=0), alpha=0.25, color=colors[ff])

colors = cm.rainbow(np.linspace(0,1,nbest))
for bb in range(nbest):
    ax.plot(xx, yy[bb,:], color=colors[bb], linestyle=':', alpha=0.75)
ax.legend()
ax.set_title('50%% and 98%% Confidence Intervals of the %d Best Samples, \nusing median among realizations' % (nbest))

Find medium, small, and large anistotropy runs among these 100 samples

#### plot_Clg0_medians

In [ ]:
def plot_Clg0_medians(Cl_best, lmax, nshow):
    fig, ax = plot.figax(figsize=(8,5), xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel='$C_{\ell>0}/C_0$')

    xx = fobs_yrs
    yy = np.sum(Cl_best[:,:,:,1:], axis=-1)/Cl_best[:,:,:,0,] # (B,F,R)
    yy = np.median(yy, axis=-1) # (B,F) median over realizations

    ax.plot(xx, np.median(yy, axis=0), color='k')
    for pp in [50, 98]:
        percs = pp/2
        percs = [50-percs, 50+percs]
        ax.fill_between(xx, *np.percentile(yy, percs, axis=0), alpha=0.2, color='k')

    colors = cm.rainbow(np.linspace(1, 0, nshow))
    for bb in range(nshow):
        ax.plot(xx, yy[bb,:], color=colors[bb], linestyle=':', alpha=0.75,
                linewidth=2)
    ax.legend()
    # ax.set_title('50%% and 98%% confidence intervals of the %d best samples \nusing realizations medians, lmax=%d'
    #             % (nbest, lmax))
    return fig
fig = plot_Clg0_medians(Cl_best, lmax, nshow=15)

In [ ]:
def plot_Clg0_llp1_medians(Cl_best, lmax, nshow):
    fig, ax = plot.figax(figsize=(8,5), xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel='$C_{\ell>0}/C_0$')

    xx = fobs_yrs
    yy = np.sum(Cl_best[:,:,:,1:], axis=-1)/Cl_best[:,:,:,0,] # (B,F,R)
    yy = np.median(yy, axis=-1) # (B,F) median over realizations

    ax.plot(xx, np.median(yy, axis=0), color='k')
    for pp in [50, 98]:
        percs = pp/2
        percs = [50-percs, 50+percs]
        ax.fill_between(xx, *np.percentile(yy, percs, axis=0), alpha=0.2, color='k')

    colors = cm.rainbow(np.linspace(1, 0, nshow))
    for bb in range(nshow):
        ax.plot(xx, yy[bb,:], color=colors[bb], linestyle=':', alpha=0.75,
                linewidth=2)
    ax.legend()
    # ax.set_title('50%% and 98%% confidence intervals of the %d best samples \nusing realizations medians, lmax=%d'
    #             % (nbest, lmax))
    return fig
fig = plot_Clg0_llp1_medians(Cl_best, lmax, nshow=15)